# Save and Retore TF models

what could potentially be saved when you are doing machine learning?

1. You can save the architecture of the model and the learned weights associated with it.
2. You might want to save some training characteristics like the loss and accuracy of the model while training or even the whole training architecture.
3. You might want to save hyperparameters and other operations to restart training later or replicate a result.

## File architecture

In TF, when you save your data the usual way, you end up with different type of files:


![Save Model Structure](./_assets_/save_model.png)


1. A `checkpoint` file: The checkpoint file is just a bookkeeping file that you can use in combination of high-level helper for loading different time saved chkp files.

2. Some `data` files: The `.data` files hold the data (weights) itself (this one is usually quite big in size). There can be many data files because they can be sharded and/or created on multiple timesteps while training. Data files are a lot heavier than the meta files.

3. A `meta` file: The `.meta` file holds the compressed Protobufs graph of your model and all the metadata associated (collections, learning rate, operations, etc.)

4. An `index` file: The .index file holds an immutable key-value table linking a serialised tensor name and where to find its data in the chkp.data files. The index file is very light as expected since it’s just a key-value table.

5. If you use `TensorBoard`, an `event` file: The events file store everything you need to visualise your model and all the data measured while you were training using summaries. This has nothing to do with saving/restoring your models itself.

**Notes:**

* The weights filename is as follow: `<prefix>-<global_step>.data-<shard_index>-of-<number_of_shards>`.
* The model has been saved 2 data files are a lot heavier than the meta files which is to be expected as they are containing the weights of our model.


### Protocal Buffers

* Protocol Buffers often abbreviated Protobufs is the format used by TF to store and transfer data efficiently.

You can use Protobufs as:

* An uncompressed, human-friendly, text format with the extension `.pbtxt`
* A compressed, machine friendly, binary format with the extension `.pb` or `no extension at all`


In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

print(tf.__version__)

# Currently, we are in the default graph scope
# Let's design some variables
v1 = tf.Variable(1. , name="v1")
v2 = tf.Variable(2. , name="v2")

# Let's design an operation
a = tf.add(v1, v2)

# We can check easily that we are indeed in the default graph
print(tf.get_default_graph())

W1208 12:28:23.997215 140734901114304 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


2.0.0


In [7]:
# Let's create a Saver object
# By default, the Saver handles every Variables related to the default graph
all_saver = tf.train.Saver()

# But you can precise which vars you want to save (as a list) and under which name (with a dict)
v2_saver = tf.train.Saver({"v2": v2})

In [10]:
save_model_dir = './_save_model_'

# By default the Session handles the default graph and all its included variables
with tf.Session() as sess:
  # Init v1 and v2   
  sess.run(tf.global_variables_initializer())
    
  # Now v1 holds the value 1.0 and v2 holds the value 2.0
  # and we can save them
  all_saver.save(sess, save_model_dir + '/data-all')

  # or saves only v2
  v2_saver.save(sess, save_model_dir + '/data-v2')